In [1]:
import numpy as np
from typing import Tuple
import pandas as pd
import pickle
import scipy.special as sc
from matplotlib import pyplot as plt
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
import math
import sys
%matplotlib notebook

In [2]:
unpickled_df = pd.read_pickle('./decays.pickle')
unpickled_df.head()

,parameters,decay
0,"[{'sigma': 1435168.9390908787, 'mu': 89.058309...","[0.20414764740139812, 0.19512088018885956, 0.1..."
0,"[{'sigma': 6273819.790735516, 'mu': 76.4296341...","[0.1853657500837078, 0.17325324056605024, 0.16..."
0,"[{'sigma': 1185609.7405302143, 'mu': 145.77550...","[0.20965104452381175, 0.20151618304390082, 0.1..."
0,"[{'sigma': 6143423.656344486, 'mu': 71.4151574...","[0.185046111522035, 0.1727535353636631, 0.1607..."
0,"[{'sigma': 6366540.795349254, 'mu': 132.191461...","[0.19235541864543546, 0.18131712610011289, 0.1..."


In [3]:
print(unpickled_df['parameters '].iloc[0])

[{'sigma': 1435168.9390908787, 'mu': 89.05830983771534, 'd': 89, 'th': 2.8163372753619518, 'th0': 6.5}]


In [4]:
def weighted_mse(u_exp: np.ndarray, u: np.ndarray, time: np.ndarray, eps: float = + 10 ** (-15)) -> Tuple[np.ndarray, np.ndarray]:
    dt = np.diff(time, n=1, axis=0)
    dt = np.append(dt, dt[-1])

    amplitude = np.dot(dt, u * u_exp) / np.dot(dt, u ** 2)
    e = (u_exp - amplitude * u) / (u_exp + eps) * dt

    loss = np.dot(e, e)
#     if (np.log(loss) == math.inf):
#         print('u_exp\n', u_exp)
#         print('u\n', u)
#         print('amplitude = ', amplitude)
#         print ('loss = ', loss)
#         sys.exit()
    return loss, amplitude



t = 1e-3 * np.array([
    0.135, 0.180, 0.235, 0.315, 0.365, 0.420,
    0.485, 0.560, 0.650, 0.750, 0.865, 1.000,
    1.155, 1.335, 1.540, 1.780, 2.055, 2.370,
    2.740, 3.160, 3.650, 4.215, 4.870, 5.625,
    6.495, 7.500, 8.660, 10.00, 11.55, 13.34,
    15.40, 17.79, 20.54, 23.72, 27.39, 31.63,
    36.52, 42.17, 48.70, 56.24, 64.94, 74.99,
    86.60, 100.0, 115.5, 133.4
])


In [5]:
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75200 entries, 0 to 0
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   parameters   75200 non-null  object
 1   decay        75200 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


## Перенесём все данные в таблицу для удобства работы

In [6]:
d = []
for p in unpickled_df['parameters ']:
    d.append({'Sigma': p[0].get('sigma'), 'mu':p[0].get('mu'),
              'd':p[0].get('d'), 'th': p[0].get('th'),
              'th0':p[0].get('th0')})
decay = []
df = pd.DataFrame(d)
dec_df = []
for id, p in enumerate(unpickled_df['decay']):
    dec_df.append(p)
df['Decay'] = dec_df
df.head(10)

,Sigma,mu,d,th,th0,Decay
0,1.435169e+06,89.058310,89,2.816337,6.5,"[0.20414764740139812, 0.19512088018885956, 0.1..."
1,6.273820e+06,76.429634,89,9.039558,6.5,"[0.1853657500837078, 0.17325324056605024, 0.16..."
2,1.185610e+06,145.775506,89,3.942288,6.5,"[0.20965104452381175, 0.20151618304390082, 0.1..."
3,6.143424e+06,71.415157,89,7.672864,6.5,"[0.185046111522035, 0.1727535353636631, 0.1607..."
4,6.366541e+06,132.191462,89,4.341253,6.5,"[0.19235541864543546, 0.18131712610011289, 0.1..."
5,2.202663e+06,67.409757,89,3.684940,6.5,"[0.19707648324855812, 0.18683864905413106, 0.1..."
6,5.452385e+06,58.463764,89,4.742516,6.5,"[0.18438771577303842, 0.17182722376412168, 0.1..."
7,8.362600e+06,27.018713,89,1.604036,6.5,"[0.1688063648017043, 0.15301738140312576, 0.13..."
8,3.133573e+06,139.161563,89,10.772532,6.5,"[0.20015195150516177, 0.19068437501823504, 0.1..."
9,5.335198e+06,47.146775,89,8.921513,6.5,"[0.18162651605649227, 0.16864305147554345, 0.1..."


## Подсчитаем корреляцию между входными признаками th, sigma,mu

In [7]:
correlations_data = df.corr()
print(correlations_data)

          Sigma        mu   d        th  th0
Sigma  1.000000  0.003973 NaN  0.002132  NaN
mu     0.003973  1.000000 NaN -0.003228  NaN
d           NaN       NaN NaN       NaN  NaN
th     0.002132 -0.003228 NaN  1.000000  NaN
th0         NaN       NaN NaN       NaN  NaN


### Из полученных значений видно, что d и th0 можно убрать из анализируемых данных. Для всех остальных переменных корреляция мала, что говорит о том что эти признаки не избыточны и мы должны их влючить в анализ

In [8]:
tmp_data = pd.DataFrame()
tmp_data = df.copy()
#tmp_data['Outside_D_Fact'] = tmp_data['d'] - 2 * tmp_data['th0'] + 2 * tmp_data['th']
#del data['d']
#del data ['th0']
tmp_data.head()

,Sigma,mu,d,th,th0,Decay
0,1.435169e+06,89.058310,89,2.816337,6.5,"[0.20414764740139812, 0.19512088018885956, 0.1..."
1,6.273820e+06,76.429634,89,9.039558,6.5,"[0.1853657500837078, 0.17325324056605024, 0.16..."
2,1.185610e+06,145.775506,89,3.942288,6.5,"[0.20965104452381175, 0.20151618304390082, 0.1..."
3,6.143424e+06,71.415157,89,7.672864,6.5,"[0.185046111522035, 0.1727535353636631, 0.1607..."
4,6.366541e+06,132.191462,89,4.341253,6.5,"[0.19235541864543546, 0.18131712610011289, 0.1..."


In [9]:
tmp_data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75200 entries, 0 to 75199
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sigma   75200 non-null  float64
 1   mu      75200 non-null  float64
 2   d       75200 non-null  int64  
 3   th      75200 non-null  float64
 4   th0     75200 non-null  float64
 5   Decay   75200 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 39.0 MB


### Разобъем выборку на тестовую и обучающую с перемешиванием

In [10]:
rs = ShuffleSplit(n_splits=1, test_size=0.15, random_state=1, train_size=0.85)
print(rs.get_n_splits(tmp_data))
print(rs)
for train_index, test_index in rs.split(tmp_data):
    print("TRAIN:", train_index, "TEST:", test_index)
print("All len = ", len(tmp_data), " and len after splitting =", len(train_index) + len(test_index))
train_data = tmp_data.iloc[train_index].copy()
test_data = tmp_data.iloc[test_index].copy()
train_data

1
ShuffleSplit(n_splits=1, random_state=1, test_size=0.15, train_size=0.85)
TRAIN: [19628  2486 64883 ... 73349 50057  5192] TEST: [15067 15441 53512 ... 17249 38159 67661]
All len =  75200  and len after splitting = 75200


,Sigma,mu,d,th,th0,Decay
19628,2.208994e+06,49.822443,89,2.721457,6.5,"[0.19375629692752577, 0.18294351872284478, 0.1..."
2486,4.145052e+06,89.330878,89,3.844767,6.5,"[0.192993357268207, 0.1820247479384618, 0.1711..."
64883,9.213975e+06,50.220776,89,6.476081,6.5,"[0.1752133606610032, 0.16091191549296152, 0.14..."
10027,9.713393e+06,133.763822,89,1.312150,6.5,"[0.18759459262951242, 0.17560025481240513, 0.1..."
51126,6.594258e+06,48.606896,89,5.393066,6.5,"[0.17945400597800243, 0.1659367937378173, 0.15..."
...,...,...,...,...,...,...
20609,5.253770e+06,69.337570,89,8.079307,6.5,"[0.18670506995857652, 0.1747114005156804, 0.16..."
21440,7.013614e+06,18.160462,89,10.886509,6.5,"[0.16552623431106728, 0.14911657811994278, 0.1..."
73349,6.281616e+06,79.829317,89,1.861306,6.5,"[0.18664879988311645, 0.17448650105585312, 0.1..."
50057,6.611047e+06,47.219636,89,6.444066,6.5,"[0.17896371071292033, 0.16537861252358166, 0.1..."


### Отнормируем обучающую выборку и запомним параметры нормировки

In [11]:
scaler = StandardScaler()
print(scaler.fit(train_data.iloc[:,0:5]))
print(scaler.mean_)
means = scaler.mean_
print(scaler.scale_)
stds = scaler.scale_
train_data.iloc[:, 0:5] = scaler.transform(train_data.iloc[:, 0:5])
train_data

StandardScaler()
[5.49793364e+06 8.00354882e+01 8.90000000e+01 6.18313815e+00
 6.50000000e+00]
[2.58728391e+06 4.04162034e+01 1.00000000e+00 3.19023924e+00
 1.00000000e+00]


,Sigma,mu,d,th,th0,Decay
19628,-1.271194,-0.747548,0.0,-1.085085,0.0,"[0.19375629692752577, 0.18294351872284478, 0.1..."
2486,-0.522897,0.229992,0.0,-0.732977,0.0,"[0.192993357268207, 0.1820247479384618, 0.1711..."
64883,1.436271,-0.737692,0.0,0.091825,0.0,"[0.1752133606610032, 0.16091191549296152, 0.14..."
10027,1.629299,1.329376,0.0,-1.526841,0.0,"[0.18759459262951242, 0.17560025481240513, 0.1..."
51126,0.423736,-0.777624,0.0,-0.247653,0.0,"[0.17945400597800243, 0.1659367937378173, 0.15..."
...,...,...,...,...,...,...
20609,-0.094371,-0.264694,0.0,0.594366,0.0,"[0.18670506995857652, 0.1747114005156804, 0.16..."
21440,0.585819,-1.530946,0.0,1.474300,0.0,"[0.16552623431106728, 0.14911657811994278, 0.1..."
73349,0.302898,-0.005101,0.0,-1.354705,0.0,"[0.18664879988311645, 0.17448650105585312, 0.1..."
50057,0.430225,-0.811948,0.0,0.081789,0.0,"[0.17896371071292033, 0.16537861252358166, 0.1..."


### По найденной дисперсии и среднему отнормируем тестовую выборку

In [12]:
test_data.iloc[:, 0:5] = scaler.transform(test_data.iloc[:, 0:5])
test_data

,Sigma,mu,d,th,th0,Decay
15067,-0.997694,0.239632,0.0,-1.224529,0.0,"[0.19710951874393856, 0.18686734402129945, 0.1..."
15441,0.702035,0.865774,0.0,1.501683,0.0,"[0.18729061178144052, 0.17607316033246334, 0.1..."
53512,-0.257575,1.301108,0.0,0.960704,0.0,"[0.19492830118742754, 0.18461371393381612, 0.1..."
55517,-0.439870,-0.182053,0.0,0.296815,0.0,"[0.18975509517206954, 0.17826619531626262, 0.1..."
18250,0.870267,-1.471396,0.0,-0.450235,0.0,"[0.16612494686759216, 0.14972489038879724, 0.1..."
...,...,...,...,...,...,...
69892,0.814175,0.781662,0.0,-0.878862,0.0,"[0.1882601086365116, 0.17643418646482018, 0.16..."
72626,-0.020505,-0.181898,0.0,-0.651101,0.0,"[0.18716586487573802, 0.17512817120621807, 0.1..."
17249,0.133407,1.380844,0.0,0.863656,0.0,"[0.1928662308439771, 0.18225947646392704, 0.17..."
38159,1.210575,-0.099357,0.0,1.416446,0.0,"[0.1803007929163741, 0.16759557921691812, 0.15..."


In [13]:
##сохранение датасета для построения графиков
df = pd.concat([train_data.copy(), test_data.copy()])
df.to_pickle('Dataset')
df

,Sigma,mu,d,th,th0,Decay
19628,-1.271194,-0.747548,0.0,-1.085085,0.0,"[0.19375629692752577, 0.18294351872284478, 0.1..."
2486,-0.522897,0.229992,0.0,-0.732977,0.0,"[0.192993357268207, 0.1820247479384618, 0.1711..."
64883,1.436271,-0.737692,0.0,0.091825,0.0,"[0.1752133606610032, 0.16091191549296152, 0.14..."
10027,1.629299,1.329376,0.0,-1.526841,0.0,"[0.18759459262951242, 0.17560025481240513, 0.1..."
51126,0.423736,-0.777624,0.0,-0.247653,0.0,"[0.17945400597800243, 0.1659367937378173, 0.15..."
...,...,...,...,...,...,...
69892,0.814175,0.781662,0.0,-0.878862,0.0,"[0.1882601086365116, 0.17643418646482018, 0.16..."
72626,-0.020505,-0.181898,0.0,-0.651101,0.0,"[0.18716586487573802, 0.17512817120621807, 0.1..."
17249,0.133407,1.380844,0.0,0.863656,0.0,"[0.1928662308439771, 0.18225947646392704, 0.17..."
38159,1.210575,-0.099357,0.0,1.416446,0.0,"[0.1803007929163741, 0.16759557921691812, 0.15..."


## PyTorch

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import sys
import torchvision
torch.backends.cudnn.enabled = False

### convert pd.Dataframe to torch.tensor

In [28]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

device = get_device()
print(device)   
print(torch.cuda.is_available())
print(torch.cuda.device_count())

cuda:0
True
1


### Тело сети

In [65]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear (6, 64)
        self.fc2 = nn.Linear (64, 64)
        self.fc3 = nn.Linear (64, 64)
        self.fc4 = nn.Linear (64, 64)
        self.fc5 = nn.Linear (64, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x =  self.fc5(x)
        return x
        

net = Net()
torch.save(net, 'modelentire.pth')
net = net.double().to(device)
print(device)
loss_function = nn.MSELoss()
#optimizer = optim.Adam(net.parameters(), lr=0.0032)
for n, p in net.named_parameters():
    print(p.device, '', n)

cuda:0
cuda:0  fc1.weight
cuda:0  fc1.bias
cuda:0  fc2.weight
cuda:0  fc2.bias
cuda:0  fc3.weight
cuda:0  fc3.bias
cuda:0  fc4.weight
cuda:0  fc4.bias
cuda:0  fc5.weight
cuda:0  fc5.bias


In [40]:
def mean_absolute_percentage_error(y_pred, y_true, eps=10**(-2)): 
    return abs(torch.mean((y_true-y_pred)/(y_true + eps)))*100

### Запуск обучения по батчам

In [41]:
def appending_to_tensor(tensor, rand_2_strs):
    tmp_tensor = torch.tensor([rand_2_strs.get('Sigma').iloc[0],rand_2_strs.get('Sigma').iloc[1],
                               rand_2_strs.get('mu').iloc[0],rand_2_strs.get('mu').iloc[1],
                               rand_2_strs.get('th').iloc[0],rand_2_strs.get('th').iloc[1],
                               np.log(weighted_mse(np.array(rand_2_strs.get('Decay').iloc[0]),np.array(rand_2_strs.get('Decay').iloc[1]),t)[0])])
#     if (np.log(weighted_mse(np.array(rand_2_strs.get('Decay').iloc[0]),np.array(rand_2_strs.get('Decay').iloc[1]),t)[0]) == math.inf):
#         print('Sigma1', rand_2_strs.get('Sigma').iloc[0])
#         print('Sigma2', rand_2_strs.get('Sigma').iloc[1])
#         print('mu1', rand_2_strs.get('mu').iloc[0])
#         print('mu2', rand_2_strs.get('mu').iloc[1])
#         print('th1', rand_2_strs.get('th').iloc[0])
#         print('th2', rand_2_strs.get('th').iloc[1])
#         print('log(mse)', np.log(weighted_mse(np.array(rand_2_strs.get('Decay').iloc[0]),np.array(rand_2_strs.get('Decay').iloc[1]),t)[0]))
# #         sys.exit()
    tmp_tensor = torch.unsqueeze(tmp_tensor, 0)
    tmp_tensor = torch.cat((tensor,tmp_tensor),0)
    return(tmp_tensor)


def create_batches_to_device(train_df, test_df, device, batch_size=128):
    train_tensor = torch.tensor([])
    test_tensor = torch.tensor([])
    for i in range (batch_size):
        rand_train_str = train_df.sample(2)
        train_tensor = appending_to_tensor(train_tensor, rand_train_str)
        rand_test_str = test_df.sample(2)
        test_tensor = appending_to_tensor(test_tensor, rand_test_str)
    test_tensor = test_tensor.to(device)
    train_tensor = train_tensor.to(device)
    trainset = torch.utils.data.DataLoader(train_tensor, batch_size=batch_size, shuffle=True)
    testset = torch.utils.data.DataLoader(test_tensor, batch_size=batch_size, shuffle=False)
    return (trainset, testset)
print(device)

cuda:0


In [42]:
# writer = SummaryWriter("../tensorboard/ex_2_LR0.0016")
# running_test_loss = 0.0
# running_train_loss = 0.0
# running_mape_loss = 0.0
# min_loss = 1000
# batch = 0
# import time
# from tqdm import tqdm
# #while (4 == 2 + 2):
# for batch in tqdm(range(350000)):
#     trainset, testset = create_batches_to_device(train_data, test_data, device)
#     for data in trainset:
#         X = data[:, 0:-1]
#         y = data[:,-1]
#         net.zero_grad()
#         output = net(X.double())
#         loss = loss_function(output.view(-1), y)
#         loss.backward()
#         optimizer.step()
#         if (math.isnan(net.state_dict()['fc1.weight'][0][0])):
#             print(y)
#             break
#     train_loss = (loss.cpu().detach().numpy())
#     for data in testset:
#         X = data[:, 0:-1]
#         y = data[:,-1]
#         output = net(X.double())
#         loss = loss_function(output.view(-1), y)
#     mape = mean_absolute_percentage_error(output.view(-1), y)
#     test_loss = (loss.cpu().detach().numpy())
#     mape_loss = (mape.cpu().detach().numpy())
#     batch = batch + 1
#     running_test_loss += test_loss
#     running_train_loss += train_loss
#     running_mape_loss += mape_loss
#     if ((batch % 100) == 99):
#         writer.add_scalars(f'loss',{
#             'training':running_train_loss / 100,
#             'test': running_test_loss / 100,
#         }, batch+1)
#         writer.add_scalar('mape loss', running_mape_loss / 100, batch+1)
#         if ((running_test_loss / 100) <  min_loss):#сохранение весов по среднему из 100 батчей
#             min_loss = running_test_loss / 100
#             torch.save(net.state_dict(), 'model_weights/model2_LR0.0016.pth') # saving model
#         running_train_loss = 0
#         running_test_loss = 0
#         running_mape_loss = 0

100%|██████████| 350000/350000 [28:27:04<00:00,  3.42it/s]   


In [66]:
writer = SummaryWriter("../tensorboard/ex_2_LR_Dynamic")
running_test_loss = 0.0
running_train_loss = 0.0
running_mape_loss = 0.0
min_loss = 1000
batch = 0
lr = 0.0032
was_loss = 100
optimizer = optim.Adam(net.parameters(), lr=lr)
while (lr > 0.00004):
    trainset, testset = create_batches_to_device(train_data, test_data, device)
    for data in trainset:
        X = data[:, 0:-1]
        y = data[:,-1]
        net.zero_grad()
        output = net(X.double())
        loss = loss_function(output.view(-1), y)
        loss.backward()
        optimizer.step()
        if (math.isnan(net.state_dict()['fc1.weight'][0][0])):
            print(y)
            break
    train_loss = (loss.cpu().detach().numpy())
    for data in testset:
        X = data[:, 0:-1]
        y = data[:,-1]
        output = net(X.double())
        loss = loss_function(output.view(-1), y)
    mape = mean_absolute_percentage_error(output.view(-1), y)
    test_loss = (loss.cpu().detach().numpy())
    mape_loss = (mape.cpu().detach().numpy())
    batch = batch + 1
    running_test_loss += test_loss
    running_train_loss += train_loss
    running_mape_loss += mape_loss
    if ((batch % 100) == 99):
        writer.add_scalars(f'loss',{
            'training':running_train_loss / 100,
            'test': running_test_loss / 100,
        }, batch+1)
        writer.add_scalar('mape loss', running_mape_loss / 100, batch+1)
        if ((running_test_loss / 100) <  min_loss):#сохранение весов по среднему из 100 батчей
            min_loss = running_test_loss / 100
            torch.save(net.state_dict(), 'model_weights/model2_LR_Dynamic.pth') # saving model
        if (((batch + 1) % 25000) == 0):
            if ((was_loss - running_test_loss / 100) < 0.1):
                lr = lr / 2
                optimizer.param_groups[0]['lr'] = lr
                print('Now lr is= ', lr, ', in batch= ', batch)
                was_loss = running_test_loss / 100
            else:
                was_loss = running_test_loss / 100
                print('lr stay. was_loss = ',was_loss)
        running_train_loss = 0
        running_test_loss = 0
        running_mape_loss = 0

lr stay. was_loss =  0.29005717505633205
Now lr is=  0.0016 , in batch=  49999
Now lr is=  0.0008 , in batch=  74999
Now lr is=  0.0004 , in batch=  99999
Now lr is=  0.0002 , in batch=  124999
Now lr is=  0.0001 , in batch=  149999
Now lr is=  5e-05 , in batch=  174999
Now lr is=  2.5e-05 , in batch=  199999
